In [1]:
import random
import numpy as np

# function to calculate the fitness of a schedule
def calculate_fitness(schedule, processing_times, setup_times):
    completion_times = [[0] * len(processing_times[0]) for _ in range(len(processing_times))]
    for i in range(len(schedule)):
        # get the machine and processing time for the job
        machine = schedule[i][0]
        processing_time = processing_times[machine][i]
        # calculate the start time of the job (maximum of completion time and setup time)
        start_time = max(completion_times[i][machine], setup_times[machine][i][i])
        # update the completion time of the job on the machine
        completion_times[i][machine] = start_time + processing_time
        # update the completion times of the remaining jobs on the machine
        for j in range(i+1, len(schedule)):
            if schedule[j][0] == machine:
                completion_times[j][machine] = completion_times[i][machine]
    # calculate the makespan (maximum completion time of all jobs on all machines)
    makespan = max(max(completion_times[i]) for i in range(len(completion_times)))
    # return the fitness (1/makespan, since we want to minimize makespan)
    return 1/makespan

# function to generate a random schedule
def generate_schedule(num_jobs, num_machines):
    schedule = []
    for i in range(num_jobs):
        machine = random.randint(0, num_machines-1)
        schedule.append((machine, i))
    return schedule

# function to update the velocity of each particle
def update_velocity(particles, best_positions, global_best_position, inertia_weight, cognitive_weight, social_weight):
    for i in range(len(particles)):
        for j in range(len(particles[i])):
            velocity = particles[i][j]['velocity']
            cognitive_component = cognitive_weight * random.random() * (best_positions[i][j]['position'] - particles[i][j]['position'])
            social_component = social_weight * random.random() * (global_best_position[j]['position'] - particles[i][j]['position'])
            new_velocity = inertia_weight * velocity + cognitive_component + social_component
            particles[i][j]['velocity'] = new_velocity

# function to update the position of each particle
def update_position(particles, processing_times, setup_times):
    for i in range(len(particles)):
        schedule = particles[i]
        for j in range(len(schedule)):
            machine = schedule[j]['position'][0]
            job = schedule[j]['position'][1]
            processing_time = processing_times[machine][job]
            if j == 0:
                setup_time = 0
            else:
                previous_job = schedule[j-1]['position'][1]
                setup_time = setup_times[machine][previous_job][job]
            start_time = max(schedule[j]['start_time'], setup_time)
            completion_time = start_time + processing_time
            particles[i][j]['start_time'] = start_time
            particles[i][j]['completion_time'] = completion_time

# function to initialize the particles
def initialize_particles(num_particles, num_jobs, num_machines):
    particles = []
    for i in range(num_particles):
        schedule = generate_schedule(num_jobs, num_machines)
        particle = []
        for j in range(num_jobs):
            machine, job = schedule[j]
            processing_time = processing_times[machine][job]
            if j == 0:
                setup_time = 0
            else:
                previous_job = schedule[j-1][1]
               


In [2]:
import numpy as np

In [2]:
import numpy as np

# Define the PSO algorithm function
def PSO(cost_function, bounds, num_particles, max_iter):
    # Initialize the particles and their velocities
    particles_pos = np.random.uniform(bounds[0], bounds[1], (num_particles, len(bounds)))
    particles_vel = np.zeros_like(particles_pos)
    
    # Initialize the global best position and cost
    global_best_pos = np.zeros(len(bounds))
    global_best_cost = np.inf
    
    # Initialize the local best positions and costs
    local_best_pos = particles_pos.copy()
    local_best_cost = np.zeros(num_particles)
    
    # Run the PSO algorithm for the specified number of iterations
    for i in range(max_iter):
        # Evaluate the cost of each particle and update the local best positions
        for j in range(num_particles):
            cost = cost_function(particles_pos[j])
            if cost < local_best_cost[j]:
                local_best_pos[j] = particles_pos[j]
                local_best_cost[j] = cost
        
        # Update the global best position and cost
        min_cost_idx = np.argmin(local_best_cost)
        if local_best_cost[min_cost_idx] < global_best_cost:
            global_best_pos = local_best_pos[min_cost_idx]
            global_best_cost = local_best_cost[min_cost_idx]
        
        # Update the velocities and positions of the particles
        for j in range(num_particles):
            r1 = np.random.uniform(size=len(bounds))
            r2 = np.random.uniform(size=len(bounds))
            particles_vel[j] = (particles_vel[j] +
                                2 * r1 * (local_best_pos[j] - particles_pos[j]) +
                                2 * r2 * (global_best_pos - particles_pos[j]))
            particles_pos[j] = particles_pos[j] + particles_vel[j]
            
            # Check if the particle position is out of bounds and correct it
            for k in range(len(bounds)):
                if particles_pos[j][k] < bounds[0][k]:
                    particles_pos[j][k] = bounds[0][k]
                    particles_vel[j][k] = 0
                elif particles_pos[j][k] > bounds[1][k]:
                    particles_pos[j][k] = bounds[1][k]
                    particles_vel[j][k] = 0
                    
    # Return the global best position and cost
    return global_best_pos, global_best_cost


In [5]:
import numpy as np

# Define the Job Shop Scheduling problem
jobs = np.array([
    [[0, 2], [1, 4], [2, 3]],
    [[1, 3], [2, 2], [0, 4]],
    [[2, 4], [0, 3], [1, 2]]
])
machines = np.array([
    [2, 1, 0],
    [1, 2, 0],
    [0, 1, 2]
])
num_jobs = jobs.shape[0]
num_machines = machines.shape[1]
num_iterations = 100
num_particles = 50
max_velocity = 5

# Define the Particle class
class Particle:
    def __init__(self):
        self.position = np.random.permutation(num_jobs)
        
        self.position = np.array(self.position, dtype=np.int32)
        self.best_position = self.position.copy()
        self.velocity = np.zeros((num_jobs, num_machines), dtype=np.int32)
        self.best_fitness = self.calculate_fitness()
#         self.position = np.random.permutation(num_jobs).astype(np.int32)
        
#         self.best_position = self.position.copy()
#         self.velocity = np.zeros((num_jobs, num_machines).astype(np.int32))
        
#         self.best_fitness = self.calculate_fitness()

    def calculate_fitness(self):
        machine_times = np.zeros((num_machines,))
        job_times = np.zeros((num_jobs,))
        for job in self.position:
            for machine in machines[job]:
                machine_times[machine] = max(machine_times[machine], job_times[job]) + jobs[job][machine][0]
                job_times[job] = machine_times[machine]
        return -np.max(job_times)

    def update_position(self):
        print('self.postion=>',self.position)
        print('\nself.velocity=>',self.velocity)
        pos = np.zeros((num_jobs, num_machines), dtype=np.int32)
        pos[np.arange(num_jobs), self.position] = 1
        print('\npos=>',pos)
        self.position = np.dot(pos + self.velocity.astype(np.int32), np.arange(num_machines))
        self.position = np.clip(self.position, 0, num_jobs - 1)
#         self.position += self.velocity.astype(np.int32).reshape(-1)
#         self.position = np.clip(self.position, 0, num_jobs-1)
#         self.position = self.position.reshape(num_jobs, 1)
#         self.position += self.velocity.astype(np.int32)
#         self.position = np.clip(self.position, 0, num_jobs - 1)
#         self.position = self.position.reshape(-1)

    def update_velocity(self, global_best_position):
        cognitive_velocity = np.random.rand() * (self.best_position - self.position)
        social_velocity = np.random.rand() * (global_best_position - self.position)
        self.velocity = self.velocity.astype(np.int32)+ cognitive_velocity + social_velocity
        self.velocity = np.clip(self.velocity, -max_velocity, max_velocity)

    def update_best_position(self):
        fitness = self.calculate_fitness()
        if fitness > self.best_fitness:
            self.best_position = self.position.copy()
            self.best_fitness = fitness

# Define the PSO function
def pso():
    particles = [Particle() for _ in range(num_particles)]
    global_best_position = particles[0].position.copy()
    global_best_fitness = particles[0].best_fitness
    for _ in range(num_iterations):
        for particle in particles:
            particle.update_velocity(global_best_position)
            particle.update_position()
            particle.update_best_position()
            if particle.best_fitness > global_best_fitness:
                global_best_position = particle.best_position.copy()
                global_best_fitness = particle.best_fitness
    return global_best_position, global_best_fitness

# Run the PSO algorithm and print the results
best_position, best_fitness = pso()
print("Best position:", best_position)
print("Best fitness:", best_fitness)


self.postion=> [2 0 1]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [1 0 2]

self.velocity=> [[ 0.70033871  0.         -0.70033871]
 [ 0.70033871  0.         -0.70033871]
 [ 0.70033871  0.         -0.70033871]]

pos=> [[0 1 0]
 [1 0 0]
 [0 0 1]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.         -0.14473518  0.14473518]
 [ 0.         -0.14473518  0.14473518]
 [ 0.         -0.14473518  0.14473518]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [0 2 1]

self.velocity=> [[ 0.31716743 -0.63433487  0.31716743]
 [ 0.31716743 -0.63433487  0.31716743]
 [ 0.31716743 -0.63433487  0.31716743]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [2 0 1]

self.velocity=> [[-0.30201066  0.          0.30201066]
 [-0.30201066  0.          0.30201066]
 [-0.30201066  0.          0.30201066]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [2 1 0]

self.velocity=> [[-0.86474864 -0.86474864  1.72949727]
 [-0.86474864 -0.86474864  1.72949727

self.postion=> [0 2 1]

self.velocity=> [[ 5.         -0.62453873 -0.62453873]
 [ 5.         -0.62453873 -0.62453873]
 [ 5.         -0.62453873 -0.62453873]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.18959634 -0.18959634]
 [ 0.          0.18959634 -0.18959634]
 [ 0.          0.18959634 -0.18959634]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 0 0]

self.velocity=> [[ 5.         -0.44676609  0.56953894]
 [ 5.         -0.44676609  0.56953894]
 [ 5.         -0.44676609  0.56953894]]

pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.89620773 -0.89620773  0.        ]
 [ 0.89620773 -0.89620773  0.        ]
 [ 0.89620773 -0.89620773  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [0 2 1]

self.velocity=> [[ 4.2460128 -0.6230064 -0.6230064]
 [ 4.2460128 -0.6230064 -0.6230064]
 [ 4.2460128 -0.6230064 -0.6230064]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0

self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.05759144 -0.05759144]
 [ 0.          0.05759144 -0.05759144]
 [ 0.          0.05759144 -0.05759144]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [1 1 1]

self.velocity=> [[-0.67435576 -1.         -0.32564424]
 [-0.67435576 -1.         -0.32564424]
 [-0.67435576 -1.         -0.32564424]]

pos=> [[0 1 0]
 [0 1 0]
 [0 1 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [0 0 0]

self.velocity=> [[2.92286174 0.94488661 0.06617704]
 [2.92286174 0.94488661 0.06617704]
 [2.92286174 0.94488661 0.06617704]]

pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.15685127 -0.15685127]
 [ 0.          0.15685127 -0.15685127]
 [ 0.          0.15685127 -0.15685127]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 0 0]

self.velocity=> [[-0.47152565  1.0452613   0.56204824]
 [-0.47152565  1.0452613   0.56204824]
 [-0.47

self.postion=> [1 1 1]

self.velocity=> [[ 2.57662489 -1.11980261 -0.45682228]
 [ 2.57662489 -1.11980261 -0.45682228]
 [ 2.57662489 -1.11980261 -0.45682228]]

pos=> [[0 1 0]
 [0 1 0]
 [0 1 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.71454325 -0.71454325]
 [ 0.          0.71454325 -0.71454325]
 [ 0.          0.71454325 -0.71454325]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.05173833 -0.05173833]
 [ 0.          0.05173833 -0.05173833]
 [ 0.          0.05173833 -0.05173833]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 0 0]

self.velocity=> [[0.29066605 0.96224599 1.63382592]
 [0.29066605 0.96224599 1.63382592]
 [0.29066605 0.96224599 1.63382592]]

pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.32943922 -0.32943922  0.        ]
 [ 0.32943922 -0.32943922  0.        ]
 [ 0.32


pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.22092449 -0.22092449  0.        ]
 [ 0.22092449 -0.22092449  0.        ]
 [ 0.22092449 -0.22092449  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [0 0 0]

self.velocity=> [[2.77710077 1.48558011 1.19405944]
 [2.77710077 1.48558011 1.19405944]
 [2.77710077 1.48558011 1.19405944]]

pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [2 2 2]

self.velocity=> [[-0.49392649 -1.1377187   0.21848908]
 [-0.49392649 -1.1377187   0.21848908]
 [-0.49392649 -1.1377187   0.21848908]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.         0.2916402 -0.2916402]
 [ 0.         0.2916402 -0.2916402]
 [ 0.         0.2916402 -0.2916402]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.82989848 -0.82989848  0.        ]
 [ 0.82989848 -0.82989848  0.        ]
 [ 0.82989848 -0.82989848  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [2 

self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 2 2]

self.velocity=> [[-0.35380541  0.06399503 -1.51820453]
 [-0.35380541  0.06399503 -1.51820453]
 [-0.35380541  0.06399503 -1.51820453]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [1 1 1]

self.velocity=> [[ 4.23755748 -1.15835445 -0.07920303]
 [ 4.23755748 -1.15835445 -0.07920303]
 [ 4.23755748 -1.15835445 -0.07920303]]

pos=> [[0 1 0]
 [0 1 0]
 [0 1 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [0 2 1]

self.velocity=> [[ 5.         -0.23413518 -0.23413518]
 [ 5.         -0.23413518 -0.23413518]
 [ 5.         -0.23413518 -0.23413518]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [

self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.75002554 -0.75002554]
 [ 0.          0.75002554 -0.75002554]
 [ 0.          0.75002554 -0.75002554]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.74242975 -0.74242975  0.        ]
 [ 0.74242975 -0.74242975  0.        ]
 [ 0.74242975 -0.74242975  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.39553053 -0.39553053]
 [ 0.          0.39553053 -0.39553053]
 [ 0.          0.39553053 -0.39553053]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 2 1]

self.velocity=> [[ 5.         -0.89585596 -0.89585596]
 [ 5.         -0.89585596 -0.89585596]
 [ 5.         -0.89585596 -0.89585596]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.0140969 -0.0140969  0.       ]
 [ 0.0140969 -0.0140969  0.       ]
 [ 0.0140969 -0.0140969  0.       ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [2 2 2]

self.velocity=> [[ 4


pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 0 0]

self.velocity=> [[2.31810944 0.18985637 0.06160329]
 [2.31810944 0.18985637 0.06160329]
 [2.31810944 0.18985637 0.06160329]]

pos=> [[1 0 0]
 [1 0 0]
 [1 0 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 2 2]

self.velocity=> [[ 4.61009965 -1.20147875  0.15664388]
 [ 4.61009965 -1.20147875  0.15664388]
 [ 4.61009965 -1.20147875  0.15664388]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.         0.2403173 -0.2403173]
 [ 0.         0.2403173 -0.2403173]
 [ 0.         0.2403173 -0.2403173]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [2 2 2]

self.velocity=> [[-2.95352428 -1.54062933 -1.12773437]
 [-2.95352428 -1.54062933 -1.12773437]
 [-2.95352428 -1.54062933 -1.12773437]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [2 2 2]

self.velocity=> [[-1.57205545 -0.55624735 -1.54043924]
 [-1.57205545 -0.556

self.postion=> [1 2 0]

self.velocity=> [[ 0.34948612 -0.34948612  0.        ]
 [ 0.34948612 -0.34948612  0.        ]
 [ 0.34948612 -0.34948612  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [2 2 2]

self.velocity=> [[-0.13547248  0.66835847  0.47218943]
 [-0.13547248  0.66835847  0.47218943]
 [-0.13547248  0.66835847  0.47218943]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [2 2 2]

self.velocity=> [[-0.51877081  0.40861796 -0.66399326]
 [-0.51877081  0.40861796 -0.66399326]
 [-0.51877081  0.40861796 -0.66399326]]

pos=> [[0 0 1]
 [0 0 1]
 [0 0 1]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.10021338 -0.10021338]
 [ 0.          0.10021338 -0.10021338]
 [ 0.          0.10021338 -0.10021338]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [1 2 0]

self.velocity=> [[ 0.31377065 -0.31377065  0.        ]
 [ 0.31377065 -0.31377065  0.        ]
 [ 0.31377065 -0.31377065  0.        ]]

pos=> [[0 1 0]
 [0 0 1]
 [1 0 0]]
self.postion=> [2 0 1]

self.veloci

self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [0 2 1]

self.velocity=> [[ 5.         -0.13122758 -0.13122758]
 [ 5.         -0.13122758 -0.13122758]
 [ 5.         -0.13122758 -0.13122758]]

pos=> [[1 0 0]
 [0 0 1]
 [0 1 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 1 0]

self.velocity=> [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

pos=> [[0 0 1]
 [0 1 0]
 [1 0 0]]
self.postion=> [2 0 1]

self.velocity=> [[ 0.          0.69580317 -0.69580317]
 [ 0.          0.69580317 -0.69580317]
 [ 0.          0.69580317 -0.69580317]]

pos=> [[0 0 1]
 [1 0 0]
 [0 1 0]]
self.postion=> [0 0 0]

self.velocity=> [[2.19106039 1.17772245 1.16438451]
 [2.19106039 1.17772245 1.16438451]
 [2.19106039 1.17772245 1.16438451]]

pos=> [[1 0 0]
 

In [15]:
jobs.shape

(3, 3, 2)

In [3]:
np.random.permutation(10)

array([8, 5, 3, 0, 6, 7, 1, 2, 9, 4])

In [4]:
np.array(np.random.permutation(10), dtype=np.int32)

array([3, 6, 2, 8, 0, 1, 7, 5, 4, 9])

In [13]:
import numpy as np
np.random.seed(0)
class Particle:
    def __init__(self, num_jobs, num_machines):
        self.position = np.random.randint(num_machines, size=num_jobs)
        print(self.position)
        self.velocity = np.zeros((num_jobs, num_machines), dtype=np.float32)
        self.best_position = np.copy(self.position)
        self.best_score = np.inf
    
    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.rand(*self.velocity.shape)
        r2 = np.random.rand(*self.velocity.shape)
        print("self.best_position=>",self.best_position.shape)
        print("position=>",self.position.shape)
        print('subv',self.best_position - self.position)
        print(c1,r1)
        cognitive = c1 * r1 * (self.best_position - self.position)
        social = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive + social
    
    def update_position(self, processing_time):
        for i in range(len(self.position)):
            job = jobs[i]
            machine = self.position[i]
            processing_time[i][machine] = max(processing_time[i][machine-1], processing_time[i-1][machine]) + job[machine]
        self.position = np.argmax(processing_time[-1], axis=0)
        self.best_score = processing_time[-1][self.position[-1]]
        if self.best_score < self.get_score(self.best_position, processing_time):
            self.best_position = np.copy(self.position)
    
    def get_score(self, position, processing_time):
        scores = np.zeros(len(position))
        for i in range(len(position)):
            job = jobs[i]
            machine = position[i]
            scores[i] = max(processing_time[i][machine-1], processing_time[i-1][machine]) + job[machine]
        return scores[-1]
    

def run_pso(num_jobs, num_machines, num_particles, max_iter):
    # Initialize particles
    particles = [Particle(num_jobs, num_machines) for _ in range(num_particles)]
    
    # Initialize global best position and score
    global_best_position = np.copy(particles[0].position)
    global_best_score = np.inf
    
    # Initialize processing time matrix
    processing_time = np.zeros((num_jobs, num_machines), dtype=np.int32)
    
    # Run PSO algorithm
    for iteration in range(max_iter):
        for particle in particles:
            particle.update_velocity(global_best_position, 0.7, 1.4, 1.4)
            particle.update_position(processing_time)
            if particle.best_score < global_best_score:
                global_best_position = np.copy(particle.best_position)
                global_best_score = particle.best_score
        
        # Reset processing time matrix
        processing_time.fill(0)
        
    return global_best_position, global_best_score


In [40]:
# Define the set of jobs
jobs = [
    [3, 5, 2],
    [2, 3, 2],
    [1, 2, 4],
    [2, 4, 3]
]

# Define the number of machines available
no_of_machines = 3

# Run the PSO algorithm
num_jobs = len(jobs)
num_particles = 20
max_iter = 100
best_position, best_score = run_pso(num_jobs, no_of_machines, num_particles, max_iter)

# Print the best position and score
print("Best position:", best_position)
print("Best score:", best_score)


[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
self.best_position=>

IndexError: arrays used as indices must be of integer (or boolean) type

In [16]:
np.zeros((5, 3), dtype=np.float32)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [18]:
pos = np.zeros((5, 3), dtype=np.float32)

In [31]:
np.random.randint(3)

1

In [33]:
for i in range(5):
    index = np.random.randint(3)
    pos[i][index] = 1

In [34]:
pos

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [39]:
import numpy as np
np.random.seed(0)
class Particle:
    def __init__(self, num_jobs, num_machines):
        self.position = np.zeros((num_jobs, num_machines), dtype=np.float32)
        for i in range(num_jobs):
            index = np.random.randint(num_machines)
            self.position[i][index] = 1
        print(self.position)
        self.velocity = np.zeros((num_jobs, num_machines), dtype=np.float32)
        self.best_position = np.copy(self.position)
        self.best_score = np.inf
    
    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.rand(*self.velocity.shape)
        r2 = np.random.rand(*self.velocity.shape)
        print("self.best_position=>",self.best_position.shape)
        print("position=>",self.position.shape)
        print('subv',self.best_position - self.position)
        print(c1,r1)
        cognitive = c1 * r1 * (self.best_position - self.position)
        social = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive + social
        print('self.velocity',self.velocity)
    
    def update_position(self, processing_time):
        for i in range(len(self.position)):
            job = jobs[i]
            machine = self.position[i]
            processing_time[i][machine] = max(processing_time[i][machine-1], processing_time[i-1][machine]) + job[machine]
        self.position = np.argmax(processing_time[-1], axis=0)
        self.best_score = processing_time[-1][self.position[-1]]
        if self.best_score < self.get_score(self.best_position, processing_time):
            self.best_position = np.copy(self.position)
    
    def get_score(self, position, processing_time):
        scores = np.zeros(len(position))
        for i in range(len(position)):
            job = jobs[i]
            machine = position[i]
            scores[i] = max(processing_time[i][machine-1], processing_time[i-1][machine]) + job[machine]
        return scores[-1]
    

def run_pso(num_jobs, num_machines, num_particles, max_iter):
    # Initialize particles
    particles = [Particle(num_jobs, num_machines) for _ in range(num_particles)]
    
    # Initialize global best position and score
    global_best_position = np.copy(particles[0].position)
    global_best_score = np.inf
    
    # Initialize processing time matrix
    processing_time = np.zeros((num_jobs, num_machines), dtype=np.int32)
    
    # Run PSO algorithm
    for iteration in range(max_iter):
        for particle in particles:
            particle.update_velocity(global_best_position, 0.7, 1.4, 1.4)
            particle.update_position(processing_time)
            if particle.best_score < global_best_score:
                global_best_position = np.copy(particle.best_position)
                global_best_score = particle.best_score
        
        # Reset processing time matrix
        processing_time.fill(0)
        
    return global_best_position, global_best_score


In [ ]:
import random

class Particle:
    def __init__(self, schedule):
        self.schedule = schedule
        self.fitness = calculate_fitness(schedule, processing_times, setup_times)
        self.velocity = [0 for _ in range(len(schedule))]

    def update_velocity(self, global_best):
        w, c1, c2 = 0.8, 2, 2  # PSO parameters
        for i in range(len(self.schedule)):
            r1, r2 = random.random(), random.random()
            cognitive = c1 * r1 * (self.best_schedule[i] - self.schedule[i])
            social = c2 * r2 * (global_best[i] - self.schedule[i])
            self.velocity[i] = w * self.velocity[i] + cognitive + social

    def update_position(self):
        for i in range(len(self.schedule)):
            self.schedule[i] += round(self.velocity[i])
        self.fitness = calculate_fitness(self.schedule, processing_times, setup_times)

def calculate_fitness(schedule, processing_times, setup_times):
    # function to calculate the fitness of a schedule
    # (same as given code)

def generate_random_particle(num_jobs, num_machines):
    schedule = generate_schedule(num_jobs, num_machines)
    particle = Particle(schedule)
    return particle

def run_pso_algorithm(num_jobs, num_machines, processing_times, setup_times, population_size=100, num_generations=100):
    # initialize the population
    population = [generate_random_particle(num_jobs, num_machines) for _ in range(population_size)]
    # set the global best particle
    global_best = population[0].schedule
    for particle in population:
        if particle.fitness > calculate_fitness(global_best, processing_times, setup_times):
            global_best = particle.schedule
    # loop through each generation
    for generation in range(num_generations):
        # update the velocity and position of each particle
        for particle in population:
            particle.update_velocity(global_best)
            particle.update_position()
            # update the particle's best position
            if particle.fitness > calculate_fitness(particle.best_schedule, processing_times, setup_times):
                particle.best_schedule = particle.schedule
            # update the global best position
            if particle.fitness > calculate_fitness(global_best, processing_times, setup_times):
                global_best = particle.schedule
    return global_best


In [ ]:
import os
from abc import ABC, abstractmethod
import numpy as np
import random
import time
import sys
import math
class SimpleTokenizer:
    '''
    Class to get the resources from the resource file
    Variables
    text -> which will contain the string from which we wnat to extract data
    separator -> these are the separator which are use in resource file to separate the data
    '''
    start = -1
    end = -1

    def __init__(self, text, separators=[' ', '\t', '\n']):
        self.text = text.replace('\r\n', '')
        self.separators = separators
        self.start = -1
        self.end = -1

    def hasToken(self):
        self.start = self.end + 1
        while (self.start < len(self.text) and self.isSeparator(self.text[self.start])):
            self.start += 1
        if (self.start < len(self.text)):
            self.end = self.start - 1
            return True
        return False

    def nextInt(self):
        while (self.start < len(self.text)):
            try:
                token = self.nextToken().strip()
                return int(token)
            except:
                pass
        raise Exception("NoSuchElement")

    def nextToken(self):
        self.skipToken()
        return self.text[self.start:self.end]

    def skipToken(self):
        self.start = self.end + 1
        while (self.start < len(self.text) and self.isSeparator(self.text[self.start])):
            self.start += 1
        self.end = self.start + 1
        while (self.end < len(self.text) and not self.isSeparator(self.text[self.end])):
            self.end += 1

    def isSeparator(self, character):
        if character in self.separators:
            return True
        return False
class Problem:
    '''
    Array Representation of the problem
    Variables:
    nJobs -> Number of Jobs
    nMachines -> Number of Machines
    processTimes -> Matrix with the process time of a job in a machine , processTimes[machine][job]
    setupTimes -> Matrix with the setup times of the next job considering the current job on same machine
    '''

    def __init__(self, instancePath):
        try:
            print(os.getcwd()+'/'+instancePath)
            with open(os.getcwd()+'/'+instancePath, 'r', newline='') as file:
                self.token = SimpleTokenizer(file.readline())
                self.nJobs = self.token.nextInt()
                self.nMachines = self.token.nextInt()
                # print('Jobs>>'+str(self.nJobs))
                
                self.processTimes = [[False for i in range(self.nJobs)] for j in range(self.nMachines)]
                
                self.setupTimes = [[[False for i in range(self.nJobs)] for j in range(self.nJobs)]for k in range(self.nMachines)]
                file.readline()
                for job in range(0, self.nJobs):
                    self.token = SimpleTokenizer(file.readline())
                    for machine in range(0, self.nMachines):
                        machineid = self.token.nextInt()
                        assert machine == machineid, "machine doesnot match ID in file"
                        self.processTimes[machine][job] = self.token.nextInt()
                file.readline()
                for machine in range(0, self.nMachines):
                    file.readline()
                    for job in range(0, self.nJobs):
                        self.token = SimpleTokenizer(file.readline())
                        for nextJob in range(0, self.nJobs):
                            self.setupTimes[machine][job][nextJob] = self.token.nextInt(
                            )
                            assert job != nextJob or self.setupTimes[machine][job][
                                nextJob] == 0, "setup between equal jobs must be zero"
        except Exception as e:
            print(e)

    def __str__(self):
        print('Array representation of Resource file')
        print('Number of Jobs => ', self.nJobs)
        print('Number of Machines => ', self.nMachines)
        print('Process Time of Jobs => ', self.processTimes)
        print('Setup Time of Jobs=>',self.setupTimes)
        return ""